<a href="https://colab.research.google.com/github/sruehr/croppingintensity/blob/main/cropintensity_SouthAmerica_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying South American cropping intensity, 2000-present
## Goal: create an annual map at 500m resolution of cropping intensity over South America

## Steps

1.  Import Zhang + 2021 cropping intensity asset (labels, 2017, 500m CI)
2.  Import MODIS temporal data for 2017 (Zhang year)
3.  Train classifier on Zhang labels using MODIS data
4.  Test performance
5.  Run classifier on additional years
6.  Calculate statistics
7.  Export annual maps as new image

## Step 1 & 2: Import Zhang labels and MODIS input feature data

In [ ]:
# Google Earth Engine Cropping Intensity Analysis
# Initial setup to load assets and MODIS data
import ee
import geemap

# Initialize Earth Engine
ee.Authenticate()  # Run this first time only
ee.Initialize(project='ee-sophieruehr')

In [ ]:
# Load MODIS land cover for agricultural masking (using 2017 to match your labels)
landcover = ee.ImageCollection("MODIS/061/MCD12Q1") \
              .filterDate('2017-01-01', '2017-12-31') \
              .first() \
              .select('LC_Type1')

# Load Zhang + 2021 cropping intensity asset (resampled modal to 500m)
intensity = ee.Image("projects/ee-sophieruehr/assets/zhang2021_southamerica_croppingintensity/Cropping_Intensity_2016_2018_500m_merged_INT")
intensity = intensity.where(intensity.eq(-1), 0)

In [ ]:
# Create masks
land_mask = landcover.neq(0)  # Remove water/no data
ag_mask = landcover.eq(12).Or(landcover.eq(14))

In [ ]:
# Load MODIS surface reflectance data for 2017 (this will be our input features)
modis = ee.ImageCollection("MODIS/061/MOD09GA") \
          .filterDate('2017-01-01', '2017-12-31') \
          .select(['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04',
                   'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07'])

print(f"Number of MODIS images in 2017: {modis.size().getInfo()}")
print(f"Intensity image bands: {intensity.bandNames().getInfo()}")

Number of MODIS images in 2017: 364
Intensity image bands: ['b1']


## Step 2: Create temporal features & training data

In [ ]:
# Calculate NDVI and other vegetation indices from MODIS
def add_indices(image):
    ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': image.select('sur_refl_b02'),
            'RED': image.select('sur_refl_b01'),
            'BLUE': image.select('sur_refl_b03')
        }).rename('EVI')
    return image.addBands([ndvi, evi])

# Apply indices calculation to all MODIS images
modis_with_indices = modis.map(add_indices)

# Create temporal features (these capture crop phenology patterns)
# Mean, std dev, max, min, and percentiles of NDVI/EVI across the year
temporal_features = modis_with_indices.select(['NDVI', 'EVI']).reduce(
    ee.Reducer.mean()
    .combine(ee.Reducer.stdDev(), sharedInputs=True)
    .combine(ee.Reducer.max(), sharedInputs=True)
    .combine(ee.Reducer.min(), sharedInputs=True)
    .combine(ee.Reducer.percentile([25, 75]), sharedInputs=True)
).reproject(crs=intensity.projection())

print(f"Temporal features bands: {temporal_features.bandNames().getInfo()}")

Temporal features bands: ['NDVI_mean', 'NDVI_stdDev', 'NDVI_max', 'NDVI_min', 'NDVI_p25', 'NDVI_p75', 'EVI_mean', 'EVI_stdDev', 'EVI_max', 'EVI_min', 'EVI_p25', 'EVI_p75']


In [ ]:
# # Function to add Day of Year (DOY) for phenology metrics
# def add_doy(image):
#     doy = ee.Date(image.get('system:time_start')).getRelative('day', 'year')
#     return image.addBands(ee.Image.constant(doy).rename('DOY').toInt16())

# # Apply to MODIS with indices
# modis_with_indices = modis.map(add_indices).map(add_doy)  # DOY is added here

# # Now compute slope
# def add_ndvi_slope(image):
#     # Safe selection
#     doy = image.select(['DOY'])
#     ndvi = image.select(['NDVI'])
#     return image.addBands(ndvi.divide(doy).rename('NDVI_slope'))

# modis_with_indices = modis_with_indices.map(add_ndvi_slope)

# # Compute seasonal stats
# ndvi_features = modis_with_indices.select(['NDVI', 'EVI', 'DOY'])

# # NDVI stats (integral, amplitude, timing of max)
# ndvi_integral = ndvi_features.select('NDVI').reduce(ee.Reducer.sum()).rename('NDVI_integral')
# ndvi_range = ndvi_features.select('NDVI').reduce(ee.Reducer.max()).subtract(
#     ndvi_features.select('NDVI').reduce(ee.Reducer.min())
# ).rename('NDVI_range')

# # Timing of peak NDVI
# max_ndvi = ndvi_features.qualityMosaic('NDVI')
# ndvi_peak_doy = max_ndvi.select('DOY').rename('NDVI_peak_DOY')

# # Same for EVI if desired
# evi_integral = ndvi_features.select('EVI').reduce(ee.Reducer.sum()).rename('EVI_integral')
# evi_range = ndvi_features.select('EVI').reduce(ee.Reducer.max()).subtract(
#     ndvi_features.select('EVI').reduce(ee.Reducer.min())
# ).rename('EVI_range')
# evi_peak_doy = ndvi_features.qualityMosaic('EVI').select('DOY').rename('EVI_peak_DOY')

# # Combine all temporal/phenology features
# phenology_features = temporal_features.addBands([
#     ndvi_integral, ndvi_range, ndvi_peak_doy,
#     evi_integral, evi_range, evi_peak_doy
# ])

# print(f"Temporal features bands: {phenology_features.bandNames().getInfo()}")

In [ ]:
# Get the projection of MODIS
temporal_features_masked = temporal_features.updateMask(land_mask).updateMask(ag_mask)
intensity_masked = intensity.updateMask(land_mask).updateMask(ag_mask)

# Combine into one image with labels
sample_image = temporal_features_masked.addBands(intensity_masked.rename('label'))

In [ ]:
print("Temporal features projection:", temporal_features_masked.projection().getInfo())
print("Intensity projection:", intensity_masked.projection().getInfo())
print("Temporal scale (meters):", temporal_features_masked.projection().nominalScale().getInfo())
print("Intensity scale (meters):", intensity_masked.projection().nominalScale().getInfo())

Temporal features projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.004311913363773704, 0, -80.00000661643446, 0, -0.0043119133637737036, 10.000405068932153]}
Intensity projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.004311913363773704, 0, -80.00000661643446, 0, -0.0043119133637737036, 10.000405068932153]}
Temporal scale (meters): 480.0000000000001
Intensity scale (meters): 480.0000000000001


In [ ]:
# Make a small test region: e.g., a bounding box in your area of interest
# Argentina
# test_region = ee.Geometry.Rectangle([-73.6, -55.0, -53.6, -21.8])  # [minLon, minLat, maxLon, maxLat]

# Medium size
test_region = ee.Geometry.Rectangle([-65, -40, -52, -25])  # [minLon, minLat, maxLon, maxLat]
# test_region = ee.Geometry.Rectangle([-61, -36, -58, -32])  # [minLon, minLat, maxLon, maxLat]

# Very small
# test_region = ee.Geometry.Rectangle([-63.0, -35.0, -62.5, -34.5])  # [minLon, minLat, maxLon, maxLat]
# test_region = ee.Geometry.Rectangle([-63, -35, -60, -33])  # [minLon, minLat, maxLon, maxLat]

# More efficient stratified sampling
def create_training_data(region, points_per_class=10000):
    # Sample all classes at once
    samples = sample_image.stratifiedSample(
        numPoints=points_per_class,
        classBand='label',
        region=region,
        scale=480,
        seed=42,
        dropNulls=True,
        tileScale=4,
        geometries=True
    )
    return samples

training_points = create_training_data(test_region, 10000)
# print(training_points.size().getInfo()) # Takes a while

In [ ]:
def stratified_split_server(fc, label_col='label', train_frac=0.7, seed=42):
    classes = fc.distinct(label_col)

    def split_class(c):
        c_value = ee.Feature(c).get(label_col)
        class_points = fc.filter(ee.Filter.eq(label_col, c_value))
        class_points = class_points.randomColumn(seed=seed)
        train_points = class_points.filter(ee.Filter.lt('random', train_frac))
        val_points = class_points.filter(ee.Filter.gte('random', train_frac))
        return ee.Dictionary({'train': train_points, 'val': val_points})

    # Map over classes
    class_list = classes.toList(classes.size())
    splits = class_list.map(split_class)

    # Merge all splits
    training_fc = ee.FeatureCollection(
        splits.map(lambda d: ee.Dictionary(d).get('train'))
    ).flatten()
    validation_fc = ee.FeatureCollection(
        splits.map(lambda d: ee.Dictionary(d).get('val'))
    ).flatten()

    return training_fc, validation_fc

# Apply the stratified split
training_data, validation_data = stratified_split_server(training_points, label_col='label', train_frac=0.7, seed=42)

In [ ]:
# # # Plot this to ensure it's working
# Map = geemap.Map()
# Map.centerObject(test_region, 7)

# vis_params = {
#     'min': 0,
#     'max': 1,
#     'palette': ['lightgray', 'yellow', 'orange', 'red']  # Non-crop to high intensity
# }
# # Add the class labels
# Map.addLayer(sample_image.select('label'), vis_params, "Class labels")

# Map.addLayer(temporal_features_masked.select('NDVI_mean'), vis_params, "NDVI")

# vis_params = {
#     'min': -1,
#     'max': 3,
#     'palette': ['lightgray', 'yellow', 'orange', 'red']  # Non-crop to high intensity
# }
# # Add the class labels
# Map.addLayer(sample_image.select('label'), vis_params, "Class labels")

# # Add the sampled training points
# # Make sure training points are styled for display
# validation_data_styled = validation_data.style(
#     color='black',
#     width=2,
#     fillColor='00000000'  # transparent fill
# )

# training_data_styled = training_data.style(
#     color='blue',
#     width=2,
#     fillColor='00000000'  # transparent fill
# )

# print(training_points.limit(5).getInfo())

# Map.addLayer(validation_data_styled, {}, "validation points")
# Map.addLayer(training_data_styled, {}, "training points")

# Map.addLayer(test_region)
# Map


In [ ]:
predictor_bands = temporal_features.bandNames()
# Sample feature values at the training points
training_samples = temporal_features.sampleRegions(
    collection=training_data,
    properties=['label'],  # target variable
    scale=480,
    geometries=True
)

# Sample feature values at the validation points
validation_samples = temporal_features.sampleRegions(
    collection=validation_data,
    properties=['label'],
    scale=480,
    geometries=True
)

## Training the RF classifier

In [ ]:
n_trees = 100
# Create the classifier
classifier = ee.Classifier.smileRandomForest(
    numberOfTrees=n_trees,
    seed=42
).train(
    features=training_samples,
    classProperty='label',
    inputProperties=predictor_bands
)
# Classify
classified_image = temporal_features_masked.classify(classifier).clip(test_region).select('classification').updateMask(land_mask).updateMask(ag_mask)

# vis_params = {
#     'min': -1,
#     'max': 3,
#     'palette': ['lightgray', 'yellow', 'orange', 'red']  # Non-crop to high intensity
# }
# Map.addLayer(classified_image, vis_params)
# Map

In [ ]:
# classified_for_export = classified_image.unmask(-9999)

# task = ee.batch.Export.image.toDrive(
#     image=classified_for_export,
#     description='RF_classification',
#     folder='EarthEngine',
#     region=test_region,
#     fileNamePrefix=f'RF_classification_ARG_tile_{n_trees}',
#     scale=480,
#     crs='EPSG:4326',
#     maxPixels=1e13,
#     formatOptions={'cloudOptimized': True},   # optional
# )

task = ee.batch.Export.image.toAsset(
    image=classified_image,
    description='RF_classification_ARG_asset',
    assetId='projects/ee-sophieruehr/assets/RF_classification_ARG',
    region=test_region,
    scale=480,
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()



In [ ]:
# classified_image_asset = ee.Image("projects/ee-sophieruehr/assets/RF_classification_ARG")

# Map = geemap.Map()
# Map.centerObject(test_region, 7)

# vis_params = {
#     'min': 0,
#     'max': 2,
#     'palette': ['black', 'lightgray','red']  # Non-crop to high intensity
# }
# # Add the class labels
# Map.addLayer(classified_image_asset, vis_params, "Class labels")
# Map

EEException: Image.load: Image asset 'projects/ee-sophieruehr/assets/RF_classification_ARG' not found (does not exist or caller does not have access).